In [115]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import copy

# Recreating the given graph
G=nx.Graph()
G.add_nodes_from(['AMS', 'BER', 'COL', 'LON', 'BRU', 'FRA','PAR', 'LYO', 'MIL', 'ROM', 'BAR', 'MAD'])

G.add_edge("AMS", "BER", time=364, cost=235)
G.add_edge("AMS", "COL", time=120, cost=40)
G.add_edge("AMS", "BRU", time=105, cost=48)
G.add_edge("BER", "AMS", time=364, cost=235)
G.add_edge("BER", "FRA", time=232, cost=125)
G.add_edge("COL", "AMS", time=120, cost=40)
G.add_edge("COL", "FRA", time=120, cost=40)
G.add_edge("FRA", "BER", time=232, cost=125)
G.add_edge("FRA", "COL", time=120, cost=40)
G.add_edge("FRA", "PAR", time=480, cost=345)
G.add_edge("FRA", "MIL", time=454, cost=240)
G.add_edge("BRU", "AMS", time=105, cost=48)
G.add_edge("BRU", "PAR", time=82, cost=80)
G.add_edge("LON", "PAR", time=136, cost=98)
G.add_edge("LON", "BRU", time=136, cost=98)
G.add_edge("PAR", "LON", time=136, cost=98)
G.add_edge("PAR", "BRU", time=82, cost=80)
G.add_edge("PAR", "FRA", time=480, cost=345)
G.add_edge("PAR", "LYO", time=112, cost=185)
G.add_edge("PAR", "BAR", time=390, cost=400)
G.add_edge("PAR", "MAD", time=225, cost=380)
G.add_edge("LYO", "PAR", time=112, cost=185)
G.add_edge("LYO", "MIL", time=176, cost=180)
G.add_edge("LYO", "BAR", time=200, cost=320)
G.add_edge("MIL", "FRA", time=454, cost=240)
G.add_edge("MIL", "ROM", time=168, cost=125)
G.add_edge("MIL", "LYO", time=176, cost=180)
G.add_edge("BAR", "PAR", time=390, cost=400)
G.add_edge("BAR", "LYO", time=200, cost=320)
G.add_edge("BAR", "MAD", time=150, cost=98)
G.add_edge("MAD", "PAR", time=225, cost=380)
G.add_edge("MAD", "BAR", time=150, cost=98)


In [116]:
def generate_path():
    node=random.choice(list(G.nodes()))
    path=[node]
    time=0
    cost=0
    unique=0
    while time < 4320: #72 hours
      # adjacent cities
      edges=list(G[node])
      # Remove unvisited nodes of cities
      unvisited_edges=[n for n in edges if n not in path]
      if not unvisited_edges:
          break
      next=random.choice(unvisited_edges)

      #get the time and cost values
      path.append(next)
      edge=G[node][next]
      time += edge['time']
      cost += edge['cost']

      # Move to the next node
      node=next

    return path, time, cost

In [117]:
def fitness_score(path):

    for i in range(len(path)-1):
      try:
        edge = G[path[i]][path[i+1]]
      except:
        return 0

    # cost of the route created
    cost = 1/sum(G[path[i]][path[i+1]]['cost'] for i in range(len(path)-1))
    unique_nodes = len(set(path))
    return cost + unique_nodes

In [118]:
# Create an individual chromosome
def create_individual():
    path, time, cost=generate_path()
    fitness=fitness_score(path)
    path_dict={'path': path, 'time': time, 'cost': cost, 'fitness': fitness}
    
    return path_dict

In [119]:
def firstGeneration(genSize):
  population=[]
  for i in range(genSize):
      path=create_individual()
      population.append(path)
  return population

In [120]:
def sorting_gen(paths):
    # fitness scores
    sorted_paths=sorted(paths, key=lambda x: x['fitness'], reverse=True)
    return sorted_paths

In [121]:
def calculate_time_and_cost(path):
    total_time=0
    total_cost=0
    for i in range(len(path)-1):

      try:
        edge=G[path[i]][path[i+1]]
      except:
        return 0, 0

      total_time += edge['time']
      total_cost += edge['cost']
    return total_time, total_cost

In [122]:
def single_point_crossover(path_1, path_2):
     # Choose a random point
    crossover_point=random.randint(1, len(path_1) - 1)
    
    # Start swapping the tails of the paths
    new_path_1=path_1['path'][:crossover_point] + path_2['path'][crossover_point:]
    new_path_2=path_2['path'][:crossover_point] + path_1['path'][crossover_point:]
    
    # Create the new individuals with with respective weights
    new_individual_1={'path': new_path_1, 'time': 0, 'cost': 0, 'fitness': 0}
    new_individual_2={'path': new_path_2, 'time': 0, 'cost': 0, 'fitness': 0}
    new_individual_1['time'], new_individual_1['cost']=calculate_time_and_cost(new_path_1)
    new_individual_2['time'], new_individual_2['cost']=calculate_time_and_cost(new_path_2)
    
    # Calculate the fitness score
    new_individual_1['fitness']=fitness_score(new_path_1)
    new_individual_2['fitness']=fitness_score(new_path_2)
    
    return new_individual_1, new_individual_2


In [123]:
 def breedPopulation(matingpool, eliteSize):
    nextGen=[]
    new_children=len(matingpool) - eliteSize

    for i in range (0, eliteSize):
      nextGen.append(matingpool[i])

    for i in range(0, new_children, 2):
      child1, child2=single_point_crossover(matingpool[i], matingpool[i+1])

      if child1['fitness'] != 0:
        nextGen.append(child1)
      
      if child2['fitness'] != 0:
        nextGen.append(child2)
    
    return nextGen

In [124]:
def mutateIndividual(path, mutation_rate):
    mutated=path
    if random.uniform(0, 1) < mutation_rate:
        # Choose two random positions 
        position1=random.randint(0, len(path['path']) - 1)
        position2=random.randint(0, len(path['path']) - 1)
        mutated['time']=0
        mutated['cost']=0
        mutated['fitness']=0
        # Swap the cities 
        path['path'][position1], path['path'][position2]=path['path'][position2], path['path'][position1]
        mutated['time'], mutated['cost']=calculate_time_and_cost(path)
        
        # Calculate the fitness score
        mutated['fitness']=fitness_score(mutated)
        mutated['fitness']=fitness_score(mutated)
    return mutated

In [125]:
def mutatePopulation(generation, mutationRate):
  mutatedGen=[]

  for i in range(0, len(generation)):
    mutatedIndividual=mutateIndividual(generation[i], mutationRate)
    mutatedGen.append(mutatedIndividual)
  return mutatedGen

In [126]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    genRanked=sorting_gen(currentGen)
    children=breedPopulation(genRanked, eliteSize)
    nextGeneration=mutatePopulation(children, mutationRate)
    return nextGeneration

In [127]:

def geneticAlgorithm(popSize, eliteSize, mutationRate, generations):
    pop=firstGeneration(popSize)   
    for i in range(generations):
      pop=nextGeneration(pop, eliteSize, mutationRate)
      print("Distance from generation ", str(i+1), ": ", str(sorting_gen(pop)[0]['fitness']))
    
    print("Final distance: " + str(sorting_gen(pop)[0]))
    bestRoute=sorting_gen(pop)[0]
    return bestRoute

In [128]:
geneticAlgorithm(popSize=1000, eliteSize=150, mutationRate=0.1, generations=1000)

Distance from generation  1 :  12.000644329896907
Distance from generation  2 :  12.000644329896907
Distance from generation  3 :  12.000644329896907
Distance from generation  4 :  11.00077399380805
Distance from generation  5 :  11.00077399380805
Distance from generation  6 :  11.00077399380805
Distance from generation  7 :  11.00077399380805
Distance from generation  8 :  11.00077399380805
Distance from generation  9 :  11.000696378830083
Distance from generation  10 :  11.000696378830083
Distance from generation  11 :  11.000696378830083
Distance from generation  12 :  11.000696378830083
Distance from generation  13 :  11.000696378830083
Distance from generation  14 :  11.000696378830083
Distance from generation  15 :  11.000696378830083
Distance from generation  16 :  11.000696378830083
Distance from generation  17 :  11.000696378830083
Distance from generation  18 :  11.000696378830083
Distance from generation  19 :  11.000696378830083
Distance from generation  20 :  11.0006963788

{'path': ['ROM',
  'MIL',
  'LYO',
  'BAR',
  'MAD',
  'PAR',
  'BRU',
  'AMS',
  'COL',
  'FRA',
  'BER'],
 'time': 1578,
 'cost': 1436,
 'fitness': 11.000696378830083}